In [1]:
def colleypy(csvpath):
    import numpy as np
    import pandas as pd
    
    #csv input should be a file with the columns "team1", "team2", "team1points", "team2points"
    games = pd.read_csv(csvpath)
    
    #from games, we fill out a team dataframe with summary stats on each unique team from the games dataframe
    teams = pd.DataFrame(columns = ['teamname', 'wins','losses','games','b'])
    teams['teamname'] = games['team1'].unique()
    
    #the next two functions fill in teams['wins'] and teams['losses']
    def calcwins(row):
        wins = games[(games['team1'] == row['teamname']) & (games['team1points'] > games['team2points'])].count()
        wins += games[(games['team2'] == row['teamname']) & (games['team2points'] > games['team1points'])].count()
        return wins

    def calclosses(row):
        losses = games[(games['team1'] == row['teamname']) & (games['team1points'] < games['team2points'])].count()
        losses += games[(games['team2'] == row['teamname']) & (games['team2points'] < games['team1points'])].count()
        return losses
    
    #the next 4 commands fill out teams using our functions and some simple math
    teams['wins'] = teams.apply(calcwins,axis=1)
    teams['losses'] = teams.apply(calclosses,axis=1)
    teams['games'] = teams['wins'] + teams['losses']
    teams['b'] = 1 + ( (teams['wins'] - teams['losses']) / 2)
    
    #declare vector of b-values, will need later
    bvector = teams['b']
    
    #create empty matrix, fill in column with team names, use team names to create columns, setting all values to zero.
    matrix = pd.DataFrame()
    matrix['team'] = teams['teamname']
    for i in teams['teamname']:
        matrix[i] = 0
    
    #dropping column one (team) to make matrix square for diagonal fill    
    matrix = matrix.drop('team',axis=1)
    
    #filling diagonal with team games played plus 2
    diag = pd.Series(data = list(teams['games']+2), index = list(teams['teamname']))
    np.fill_diagonal(matrix.values,diag)
    
    #recreating the team column as column one
    matrix.insert(0, 'team', games['team1'].unique())
    
    #make the just recreated column the index instead of a column
    matrix = matrix.set_index('team')
    
    #build dictionary where each team has a key and each key contains a list of opponents that team has played (duplicates possible)
    opponents = {}
    for team in games['team1'].unique():
        opponents[team] = []
        teamsgames = games[(games['team1'] == team) | (games['team2'] == team)]
        opps = list(teamsgames[teamsgames['team1'] != team]['team1'])
        opps.extend(list(teamsgames[teamsgames['team2'] != team]['team2']))
        opponents[team] = opps
    
    #for every team contained in the above dictionary, we take their list of opponents to set the cross point of those two
    #teams in the matrix to its current value minus 1
    for team in opponents:
        for oppo in opponents[team]:
            matrix[team][oppo] -= 1
    
    #invert the matrix for solving       
    matrix_inverse = pd.DataFrame(np.linalg.pinv(matrix.values), matrix.columns, matrix.index)
    
    #solve equation ( matrix(ranking) = bvector ) via ( matrix^-1(b) = rankings)
    ranking = matrix_inverse.dot(bvector.values)
    
    #build DataFrame for final output and fill
    output = pd.DataFrame()
    output['team'] = teams['teamname']
    output['colleyrating'] = ranking.values
    output = output.sort_values(by=['colleyrating'],ascending=False)
    
    #print and return output
    print(output)
    return output

In [2]:
import numpy as np
import pandas as pd
op = colleypy(r'C:\Users\jmccoy\Documents\Personal Docs\McCoy Matrix CFB\colleything.csv')

                  team  colleyrating
40          Ohio State      1.029458
55                 LSU      0.964445
68             Clemson      0.897372
7              Memphis      0.868056
77            Oklahoma      0.862770
46             Georgia      0.840642
109         Notre Dame      0.834664
119         Penn State      0.832072
130          Wisconsin      0.830073
18         Boise State      0.826428
10              Oregon      0.811263
39            Michigan      0.802051
94   Appalachian State      0.791476
28                Utah      0.790116
32          Cincinnati      0.789909
47             Florida      0.783225
76              Baylor      0.783188
54              Auburn      0.776176
17                 SMU      0.771176
118               Navy      0.767514
12                Iowa      0.760456
123          Minnesota      0.760250
51             Alabama      0.753456
102          Air Force      0.735807
98                 USC      0.695476
89    Florida Atlantic      0.678018
8

In [3]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)

In [4]:
print_full(op)

                      team  colleyrating
40              Ohio State      1.029458
55                     LSU      0.964445
68                 Clemson      0.897372
7                  Memphis      0.868056
77                Oklahoma      0.862770
46                 Georgia      0.840642
109             Notre Dame      0.834664
119             Penn State      0.832072
130              Wisconsin      0.830073
18             Boise State      0.826428
10                  Oregon      0.811263
39                Michigan      0.802051
94       Appalachian State      0.791476
28                    Utah      0.790116
32              Cincinnati      0.789909
47                 Florida      0.783225
76                  Baylor      0.783188
54                  Auburn      0.776176
17                     SMU      0.771176
118                   Navy      0.767514
12                    Iowa      0.760456
123              Minnesota      0.760250
51                 Alabama      0.753456
102             